In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

## Data preparation

In [ ]:
os.chdir('data/dogs-vs-cats')
%pwd

In [ ]:
# Using the script below, we'll organize our data into train, validation, and test sets.
# We'll do this by moving subsets of the data into sub-directories for each separate data set.
dirs = ['train/dog', 'train/cat', 'valid/dog', 'valid/cat', 'test/dog', 'test/cat']

if not os.path.isdir(dirs[0]):
    for d in dirs:
        os.makedirs(d)

    for c in random.sample(glob.glob('cat*'), 500):
        shutil.move(c, 'train/cat')
    for c in random.sample(glob.glob('dog*'), 500):
        shutil.move(c, 'train/dog')
    for c in random.sample(glob.glob('cat*'), 100):
        shutil.move(c, 'valid/cat')
    for c in random.sample(glob.glob('dog*'), 100):
        shutil.move(c, 'valid/dog')
    for c in random.sample(glob.glob('cat*'), 50):
        shutil.move(c, 'test/cat')
    for c in random.sample(glob.glob('dog*'), 50):
        shutil.move(c, 'test/dog')

os.chdir('../../')

In [ ]:
%pwd

In [ ]:
train_path = 'data/dogs-vs-cats/train'
test_path = 'data/dogs-vs-cats/test'
valid_path = 'data/dogs-vs-cats/valid'

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['cat','dog'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['cat','dog'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['cat','dog'], batch_size=10, shuffle=False)

# shuffle=False only for test_batches because, later when we plot the evaluation results from the model to a confusion
# matrix, we'll need to able to access the unshuffled labels for the test set. By default, the data sets are shuffled.

In [ ]:
assert train_batches.n == 1000
assert valid_batches.n == 200
assert test_batches.n == 100
assert train_batches.num_classes == valid_batches.num_classes == test_batches.num_classes == 2

In [ ]:
# generates a batch of images and labels from the training set.
# the size of this batch is determined by the batch_size we set when we created train_batches.
imgs, labels = next(train_batches)

In [ ]:
# plot the processed images within our Jupyter notebook.
# https://deeplizard.com/learn/video/LhEMXbjGV_4

def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(imgs)
print(labels)

## Build and train a CNN

In [ ]:
model = Sequential([
    Conv2D( # creates a convolution kernel that is wind with layers input which helps produce a tensor of outputs
        # no. of filters that convolutional layers will learn from.
        filters=32, # determines the number of output filters in the convolution
        kernel_size=(3,3), # determines the dimensions of the kernel. (1, 1), (3, 3), (5, 5), or (7, 7) are mostly used.
        activation='relu',
        padding='same',
        input_shape=(224,224,3) # shape of input data, equals to target size
    ),
    MaxPool2D(pool_size=(2,2), strides=2), # strides: an integer or tuple/list of 2 integers,
    # specifying the "step" of the convolution along with the height and width of the input volume.
    Conv2D(
        filters=64, # Layers early in the network architecture (i.e., closer to the actual input image) learn fewer convolutional
            # filters while layers deeper in the network (i.e., closer to the output predictions) will learn more filters.
        kernel_size=(3,3),
        activation='relu',
        padding='same'
    ),
    MaxPool2D(pool_size=(2,2), strides=2),
    Flatten(),
    Dense(units=2, activation='softmax')
])
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=10, verbose=2)

## Predict

In [ ]:
test_imgs, test_labels = next(test_batches)
plotImages(test_imgs)
print(test_labels)

In [ ]:
test_batches.classes

In [ ]:
predictions = model.predict(x=test_batches, verbose=0)
np.round(predictions[:10])

In [ ]:
# https://deeplizard.com/learn/video/bfQBPNDy5EM
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))
cm_plot_labels = ['cat','dog']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

In [ ]:
test_batches.class_indices

## Build fine-tuned VGG16 model

In [ ]:
# Download model with Internet connection
vgg16_model = tf.keras.applications.vgg16.VGG16()
vgg16_model.summary()

In [ ]:
type(vgg16_model)

In [ ]:
model = Sequential()
# Add all layers of vgg16 except last one (which has 1000 classes)
for layer in vgg16_model.layers[:-1]:
    model.add(layer)
model.summary()

In [ ]:
# This freezes the weights and other trainable parameters, weights and biases in each layer so that
# they will not be trained or updated when we later pass in our images of cats and dogs.
# because vgg16 has already learned features of cats and dogs in its original training
for layer in model.layers:
    layer.trainable = False

In [ ]:
model.add(Dense(units=2, activation='softmax'))
model.summary()

In [ ]:
model.layers[0].trainable, model.layers[-2].trainable, model.layers[-1].trainable

### Train the fine-tuned vgg16 model

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=5, verbose=2)

In [ ]:
assert model.history.history.get('accuracy')[-1] > 0.95

In [ ]:
model.save('custom_vgg16_model.h5')